In [3]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
%matplotlib inline


In [4]:
data = pd.read_csv('./data/data.csv');
data.head()

f = pd.DataFrame();
f.head()

f['title'] = data['title']
f['tags'] = data['tags']

f.head()

f.to_json('./json-1.json', orient='records', force_ascii=False)

In [5]:
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
import numpy as np
np.random.seed(2018)
import nltk
nltk.download('wordnet')
stemmer = SnowballStemmer('russian')

import json
from html.parser import HTMLParser

class MLStripper(HTMLParser):
    def __init__(self):
        self.reset()
        self.strict = False
        self.convert_charrefs= True
        self.fed = []
    def handle_data(self, d):
        self.fed.append(d)
    def get_data(self):
        return ''.join(self.fed)

def strip_tags(html):
    s = MLStripper()
    s.feed(html)
    return s.get_data()

def standardize_text(df, text_field):
    df[text_field] = df[text_field].apply(strip_tags)
    df[text_field] = df[text_field].str.replace(r"http\S+", "")
    df[text_field] = df[text_field].str.replace(r"http", "")
    df[text_field] = df[text_field].str.replace(r"[^а-яА-Я0-9(),!?@\'\`\"\_\n]", " ")
    df[text_field] = df[text_field].str.lower()
    return df

def lemmatize_stemming(text):
    return stemmer.stem(WordNetLemmatizer().lemmatize(text, pos='v'))
def preprocess(text):
    result = []
    for token in gensim.utils.simple_preprocess(text):
        if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 3:
            result.append(lemmatize_stemming(token))
    return result



[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/happylol/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [6]:
data2 = data.copy();

standardize_text(data2, "text")
standardize_text(data2, "title")

data2.head()

,text,title,tags
0,никто не может остаться равнодушным к венеции ...,письма из венеции,"[ ""выставка"" ]"
1,дарит возможности!!! в э...,дарит возможности!!! в ...,"[ ""тренировка"", ""фитнес"", ""кроссфит"" ]"
2,бесплатная открытая тренировка по ! ...,бесплатная открытая тренировка по ! ...,"[ ""фитнес"", ""тренировка"", ""кроссфит "" ]"
3,"хотите работать в германии, но не владеете ...","хотите работать в германии, но не владеете...","[ ""вебинар"", ""германия"", ""работа"" ]"
4,внимание!!! открытое занятие по и ...,внимание!!! открытое занятие по и ...,"[ ""фитнес"", ""тренировка"", ""trx"" ]"


In [7]:
def tokenize(x):
#     print(len(x.text.split(" ")))
    text = x['text'] + ' ' + x['title'] + ' '.join(x['tags']);
    return preprocess(text)

def preprocessTags(string):
    arr = json.loads(string)
    return preprocess(' '.join(arr));

data2['tags'] = data2['tags'].apply(preprocessTags);
data2['tokens'] = data2.apply(tokenize, axis=1);
data2.head()

,text,title,tags,tokens
0,никто не может остаться равнодушным к венеции ...,письма из венеции,[выставк],"[никт, может, оста, равнодушн, венец, город, т..."
1,дарит возможности!!! в э...,дарит возможности!!! в ...,"[тренировк, фитнес, кроссф]","[дар, возможн, суббот, декабр, состо, открыт, ..."
2,бесплатная открытая тренировка по ! ...,бесплатная открытая тренировка по ! ...,"[фитнес, тренировк, кроссф]","[бесплатн, открыт, тренировк, приход, проб, до..."
3,"хотите работать в германии, но не владеете ...","хотите работать в германии, но не владеете...","[вебинар, герман, работ]","[хот, работа, герман, владеет, достаточн, коли..."
4,внимание!!! открытое занятие по и ...,внимание!!! открытое занятие по и ...,"[фитнес, тренировк]","[вниман, открыт, занят, набор, дневн, групп, с..."


In [8]:
lst_col = 'tags'
data3 = pd.DataFrame({
    col:np.repeat(data2[col].values, data2[lst_col].str.len())
    for col in data2.columns.difference([lst_col])
    }).assign(**{lst_col:np.concatenate(data2[lst_col].values)})[data2.columns.tolist()]

data3.head()

,text,title,tags,tokens
0,никто не может остаться равнодушным к венеции ...,письма из венеции,выставк,"[никт, может, оста, равнодушн, венец, город, т..."
1,дарит возможности!!! в э...,дарит возможности!!! в ...,тренировк,"[дар, возможн, суббот, декабр, состо, открыт, ..."
2,дарит возможности!!! в э...,дарит возможности!!! в ...,фитнес,"[дар, возможн, суббот, декабр, состо, открыт, ..."
3,дарит возможности!!! в э...,дарит возможности!!! в ...,кроссф,"[дар, возможн, суббот, декабр, состо, открыт, ..."
4,бесплатная открытая тренировка по ! ...,бесплатная открытая тренировка по ! ...,фитнес,"[бесплатн, открыт, тренировк, приход, проб, до..."


In [9]:
from sklearn.feature_extraction.text import CountVectorizer

corpus = []
for row in data3['tokens']:
    corpus.append(" ".join(row))
    
vectorizer = CountVectorizer()

X = vectorizer.fit_transform(corpus)
X = pd.DataFrame(X.toarray())
X.shape

(137, 2563)

In [8]:
from sklearn import preprocessing

le = preprocessing.LabelEncoder()
le.fit(data3['tags'])

list(le.classes_)

Y = le.transform(data3['tags'])

In [41]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.20)

print('#train:', X_train.shape, y_train.shape, '#test:', X_test.shape, y_test.shape)

#train: (109, 2563) (109,) #test: (28, 2563) (28,)


In [45]:
from sklearn.linear_model import LogisticRegression

lf = LogisticRegression(random_state=2018).fit(X_train, y_train)
pred = lf.predict(X_test)

/Users/happylol/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/happylol/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


In [46]:
from sklearn.metrics import accuracy_score

accuracy_score(y_test, pred)

0.14285714285714285

In [47]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification

# X, y = make_classification(n_samples=1000, n_features=4,
#                            n_informative=2, n_redundant=0,
#                            random_state=0, shuffle=False)
clf = RandomForestClassifier(n_estimators=100, max_depth=2,
                             random_state=0)
clf.fit(X_train, y_train)

# print(clf.feature_importances_)

rfc_prediction = clf.predict(X_test)
accuracy_score(rfc_prediction, y_test)

0.07142857142857142

In [50]:
from sklearn import svm, datasets
from sklearn.model_selection import GridSearchCV
iris = datasets.load_iris()
parameters = {'kernel':('linear', 'rbf'), 'C':[1, 10]}
svc = svm.SVC(gamma="scale")
clf = GridSearchCV(svc, parameters, cv=5)
clf.fit(X_train, y_train)
                            



sorted(clf.cv_results_.keys())
clf_prediction = clf.predict(X_test)
accuracy_score(rfc_prediction, y_test)
                            



/Users/happylol/anaconda3/lib/python3.6/site-packages/sklearn/model_selection/_split.py:652: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of members in any class cannot be less than n_splits=5.
  % (min_groups, self.n_splits)), Warning)
/Users/happylol/anaconda3/lib/python3.6/site-packages/sklearn/model_selection/_search.py:841: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


0.07142857142857142

In [55]:
dictionary = gensim.corpora.Dictionary(data3['tokens'])
# dictionary.filter_extremes(no_below=15, no_above=0.5, keep_n=100000)
dictionary.save_as_text('./dict')


bow_corpus = [dictionary.doc2bow(doc) for doc in data3['tokens']]
# bow_corpus_tags = [dictionary.doc2bow(doc) for doc in data2['tags']]
print(len(bow_corpus))
e = pd.DataFrame(bow_corpus)
e.head()
# len(bow_corpus)

# list(le.inverse_transform(l['tags']))

137


,0,1,2,3,4,5,6,7,8,9,...,213,214,215,216,217,218,219,220,221,222
0,"(0, 1)","(1, 2)","(2, 1)","(3, 1)","(4, 1)","(5, 1)","(6, 6)","(7, 2)","(8, 1)","(9, 1)",...,None,None,None,None,None,None,None,None,None,None
1,"(14, 1)","(58, 2)","(106, 2)","(107, 2)","(108, 2)","(109, 2)","(110, 2)","(111, 1)","(112, 1)","(113, 1)",...,None,None,None,None,None,None,None,None,None,None
2,"(14, 1)","(58, 2)","(106, 2)","(107, 2)","(108, 2)","(109, 2)","(110, 2)","(111, 1)","(112, 1)","(113, 1)",...,None,None,None,None,None,None,None,None,None,None
3,"(14, 1)","(58, 2)","(106, 2)","(107, 2)","(108, 2)","(109, 2)","(110, 2)","(111, 1)","(112, 1)","(113, 1)",...,None,None,None,None,None,None,None,None,None,None
4,"(14, 1)","(58, 2)","(106, 2)","(107, 2)","(110, 1)","(111, 1)","(114, 1)","(115, 1)","(116, 1)","(117, 3)",...,None,None,None,None,None,None,None,None,None,None


In [19]:
import fasttext
data3.head()
d4 = data3.copy();

d4['tags'] = '_label_' + d4['tags'].astype(str)
d4.head()

# print(X_test)

d4.to_csv(path_or_buf='data.test.txt', columns=['text'])
d4.to_csv(r'data2.test.txt', columns=['text'], header=None, index=None, sep=' ', mode='a')
classifier = fasttext.supervised('data2.train.txt','model', label_prefix='_label_');
# model.predict('data.test.txt')
result = classifier.test('data2.test.txt')
print( 'P@1:', result.precision)
print ('R@1:', result.recall)
print ('Number of examples:', result.nexamples)
# classifier = fasttext.supervised('data.train.txt', 'model')

P@1: 0.0072992700729927005
R@1: 0.0072992700729927005
Number of examples: 137
